In [1]:
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

import torch
import torch.nn as nn
import torch.optim as optim


# params

In [2]:
v_min = -5; v_max = 5
a_min  = 0.1; a_max = 5

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
class ChoiceNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ChoiceNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()

        self.fc2 = nn.Linear(hidden_size, 1)
        # Removed the sigmoid layer

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        # Do not apply sigmoid here
        return out  # Return raw logits

all_loss = []

# NN params
input_size = 2
hidden_size = 20
learning_rate = 1e-3
num_epochs = int(500e3)
batch_size = 100

model = ChoiceNet(input_size, hidden_size).to(device)  # Move model to GPU
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1000, verbose=True)

# Training loop
for epoch in range(num_epochs):
    # Generate inputs directly on the GPU
    v_samples = (v_max - v_min) * torch.rand(batch_size, device=device) + v_min
    a_samples = (a_max - a_min) * torch.rand(batch_size, device=device) + a_min
    inputs = torch.stack([v_samples, a_samples], dim=1)

    # Compute targets
    targets = (1 - ( 1 - torch.exp( -v_samples*a_samples ) ) / ( torch.exp( v_samples*a_samples ) - torch.exp( - v_samples*a_samples ) )).unsqueeze(1)
    targets[torch.isnan(targets)] = 0.5 # if v = 0, then targets = 0/0, limit is 0.5

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    all_loss.append(loss.item())

    # loss.item is nan, stop
    if np.isnan(loss.item()):
        print(f'Loss is nan at epoch {epoch}')
        break
    # Optional: Print progress
    # if epoch % 1000 == 0:
    #     print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final loss = {loss.item():.4f}')


Using device: cuda
Final loss = 0.2228


cuda took 3.5 min?